# Prerequisites
- Homomorphic encryption pdf
- Modular arithmetic

# Theory

**Key generation**
- Choose large primes $p, q$ s.t $\gcd(p\cdot q, (p-1)(q-1)) = 1$
- Let 
    - $n = pq$
    - $\lambda = \text{lcm}(p-1, q-1)$
    
- Select random $g \in \mathbb{Z}^*_{n^2}$ 
- ensure $n$ divides $|g|$ by checking if $\mu = (L(g^\lambda \bmod n^2))^{-1} \bmod n$ exists where $L = \dfrac {x-1} n$ where division represents the quotient 
- *Public (encryption) key:* $(n,g)$
- *Private(decryption) key:* $(\lambda,\mu)$

**Encryption**
- Let $m \in \mathbb{Z}_n$ be a message
- Let $r \in \mathbb{Z}_n^*$ be a random number with $\gcd(r, n) = 1$
- Return $c = g^m \cdot r^n \bmod n^2$

**Decryption**
- $m = L(c^\lambda \bmod n^2) \cdot \mu \bmod n$

## Homomorphic properties

Let there be 2 encrypted messages
- $c_1 = E(m_1,pk) = g^{m_1} \cdot r^{n}_1 \bmod n^2$
- $c_2 = E(m_2,pk) = g^{m_2} \cdot r^{n}_2 \bmod n^2$

### Addition
> The *product* of the ciphertext decrypts to the *sum* of the plaintexts
> $$D(c_1 \cdot c_2 \bmod n^2) = m_1 + m_2 \bmod n$$

**Proof**

$\begin{equation}
\begin{split}
c_1 \cdot c_2  & = (g^{m_1} \cdot r^{n}_1)(g^{m_2} \cdot r^{n}_2) \bmod n^2 \\
& = g^{m_1 + m_2}(r_1r_2)^n \bmod n^2 \\
& = E(m_1 + m_2, pk) 
\end{split}
\end{equation}$

### Multiplication

> An encrypted plaintext raised to the power of another plaintext will decrypt to the product of the two plaintexts,
> $$D(c_1^{m_2} \bmod n^2) = m_1\ m_2 \bmod n \\ D(c_2^{m_1} \bmod n^2) = m_1\ m_2 \bmod n$$

**Proof**

$\begin{equation}
\begin{split}
c_1 \cdot c_2  & = (g^{m_1} \cdot r^{n}_1)^{m_2} \bmod n^2 \\
& = g^{m_1m_2}(r_1^{m_2})^n \bmod n^2 \\
& = E(m_1m_2, pk) 
\end{split}
\end{equation}$

# Code

In [1]:
import random
from Crypto.Util.number import GCD, bytes_to_long, getPrime, inverse, long_to_bytes

In [2]:
def keygen():
    p = getPrime(1024)
    q = getPrime(1024)
    assert GCD(p * q, (p - 1) * (q - 1)) == 1, "GCD != 1"

    n = p * q
    phi = (p - 1) * (q - 1)
    lambdaa = phi
    g = n + 1
    mu = inverse(phi, n)

    return (n, g), (lambdaa, mu)


def encrypt(m, n, g):
    while True:
        r = random.randint(1, n - 1)
        if GCD(n, r) == 1:
            break
    c = (pow(g, m, n**2) * pow(r, n, n**2)) % (n**2)

    return c


def decrypt(c, n, g, lambdaa, mu):
    temp = pow(c, lambdaa, n**2)
    temp = (temp - 1) // n
    m = (temp * mu) % n

    return m

In [3]:
(n, g), (lambdaa, mu) = keygen()
# (n, g), (lambdaa, mu)

In [4]:
# Encryption

m = bytes_to_long(b"secret_message")
c = encrypt(m, n, g)
m_decr = decrypt(c, n, g, lambdaa, mu)

print(long_to_bytes(m_decr))

b'secret_message'


In [5]:
# Addition
(n, g), (lambdaa, mu) = keygen()

m1 = 1000
m2 = 2000

c1 = encrypt(m1, n, g)
c2 = encrypt(m2, n, g)

c3 = c1 * c2
m3 = decrypt(c3, n, g, lambdaa, mu)

m1 + m2, m3

(3000, 3000)

In [6]:
# Multiplication

(n, g), (lambdaa, mu) = keygen()

m1 = 1000
m2 = 2000

c1 = encrypt(m1, n, g)

c3 = pow(c1, m2, n**2)
m3 = decrypt(c3, n, g, lambdaa, mu)

m1 * m2, m3

(2000000, 2000000)

# Resources

- [Wikipedia](https://en.wikipedia.org/wiki/Paillier_cryptosystem)
- [Opendmined blog](https://blog.openmined.org/the-paillier-cryptosystem/)
- [original paper](https://www.cs.tau.ac.il/~fiat/crypt07/papers/Pai99pai.pdf )
- Paillier_cryptosystemhttp://cryptowiki.net/index.php?title=Partially_homomorphic_encryption_schemes